In [1]:
import sys
import os
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
  
# append the path of the parent directory
sys.path.append("..")

C:\Users\thomas.bush\AppData\Local\Temp\ipykernel_31692\2290338610.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Helper Functions

In [ ]:
def load_metadata(root_path):
    """Load metadata from JSON files into a dict of dicts.

    Args:
        root_path (str): The root directory containing model subdirectories.

    Returns:
        dict: A dictionary with model IDs as keys and metadata dictionaries as values.
    """
    metadata_dict = {}
    for model_dir in os.listdir(root_path):
        model_path = os.path.join(root_path, model_dir)
        if os.path.isdir(model_path):
            metadata_file = os.path.join(model_path, f"{model_dir}_metadata.json")
            if os.path.isfile(metadata_file):
                with open(metadata_file, 'r') as file:
                    metadata_dict[model_dir] = json.load(file)
    return metadata_dict

Exploration / Visualisation Functions


In [ ]:
def plot_resolution_histogram(metadata, resolution_axis):
    """Plot a histogram of the resolutions in the metadata for a specified axis.

    Args:
        metadata (dict): A dictionary with model IDs as keys and metadata dictionaries as values.
        resolution_axis (str): The axis to plot the resolution histogram for ('x' or 'y').
    """
    # Ensure that the resolution_axis is either 'x' or 'y'
    if resolution_axis not in ('x', 'y'):
        raise ValueError("resolution_axis must be 'x' or 'y'")

    # Extract the resolutions for the specified axis
    resolutions = [model_data[f'grid_resolution_{resolution_axis}'] for model_data in metadata.values()]

    # Plot histogram for the specified resolutions
    plt.figure(figsize=(7, 5))
    plt.hist(resolutions, bins=20, color='blue' if resolution_axis == 'x' else 'green', edgecolor='black')
    plt.title(f'Histogram of {resolution_axis.upper()} Resolutions')
    plt.xlabel('Resolution')
    plt.ylabel('Frequency')
    plt.show()

def find_different_resolutions(metadata):
    """Find models where X and Y resolutions are different.

    Args:
        metadata (dict): A dictionary with model IDs as keys and metadata dictionaries as values.

    Returns:
        list: A list of model IDs where the X and Y resolutions are not the same.
    """
    different_resolutions = [model_id for model_id, model_data in metadata.items()
                             if model_data['grid_resolution_x'] != model_data['grid_resolution_y']]
    return different_resolutions

In [ ]:
root_dir = r"C:\Users\thomas.bush\repos\dyna-landslide-surrogate\data"

metadata = load_metadata(root_dir)

In [ ]:
# Find and print the list of models with different X and Y resolutions
models_with_different_resolutions = find_different_resolutions(metadata)
print("Models with different X and Y resolutions:", models_with_different_resolutions)

In [ ]:
# Plot the histogram of X resolutions
plot_resolution_histogram(metadata, 'x')

# Plot the histogram of Y resolutions
plot_resolution_histogram(metadata, 'y')